In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("mtsamples.csv" , usecols=['description', 'keywords', "transcription" , "medical_specialty","sample_name", "medical_specialty"])
df.size

24995

In [4]:
df.dropna(axis = 0 , inplace = True)

In [5]:
df["total_description"] = "description : " + df["description"] + "medical_specialty : " + df["medical_specialty"] + " sample_name : " +  df["sample_name"] + "transcription : " + df["transcription"]

In [6]:
import tensorflow as tf
from transformers import BartTokenizer, TFBartForConditionalGeneration
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the optimizer and loss function
optimizer = Adam(learning_rate=5e-5)
loss = SparseCategoricalCrossentropy(from_logits=True)

In [7]:
import tensorflow.compat.v1 as tf

# ... rest of your code

with tf.compat.v1.Session() as sess:
    # Code requiring the default graph
    graph = tf.compat.v1.get_default_graph()
# Load pre-trained BART model
model = TFBartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Load BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')



All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [8]:
max_input_length = 800
max_output_length = 80

# Define the batch size and number of epochs for training
batch_size = 4
epochs = 4

In [9]:
def tokenize_inputs_and_outputs(inputs, outputs):
    input_ids = []
    attention_masks = []
    decoder_input_ids = []
    labels = []
    for i in range(len(inputs)):
        input_text = inputs[i]
        output_text = outputs[i]
        input_tokenized = tokenizer.encode(input_text, max_length=max_input_length, padding='max_length', truncation=True, return_tensors='tf')
        output_tokenized = tokenizer.encode(output_text, max_length=max_output_length, padding='max_length', truncation=True, return_tensors='tf')
        input_ids.append(input_tokenized[0])
        #attention_masks.append(input_tokenized['attention_mask'][0])
        #decoder_input_ids.append(output_tokenized['input_ids'][0])
        labels.append(output_tokenized[0])
    return {'input_ids': tf.stack(input_ids),
           # 'attention_mask': tf.stack(attention_masks),
            #'decoder_input_ids': tf.stack(decoder_input_ids),
            'labels': tf.stack(labels)}

In [10]:
import re
def clean_highlight(text):
    newText = text.lower()
    newText = re.sub('[^w s d .]','',newText)
    newText = ' '.join(newText.split())
    newText = 'START '+ newText + ' END'
    return newText
def clean_body(text):
    #newText = text[3:]
    newText = text.lower()
    newText = re.sub('[^w s d .]','',newText)
    newText = ' '.join(newText.split())
    tokens = [w for w in newText.split() ]
    long_words=[]
    for i in tokens:
        if len(i)>=3:
            long_words.append(i)   
    return ( " ".join(long_words)).strip()

In [11]:
# Create a dataset from the input and output data
class SummaryDataset(tf.keras.utils.Sequence):
    def __init__(self, inputs, outputs, batch_size):
        self.inputs = inputs
        self.outputs = outputs
        self.batch_size = batch_size
        self.indexes = np.arange(len(inputs))

    def __len__(self):
        # If using only training data, return the number of training batches:
        return len(self.inputs) // self.batch_size
    def __getitem__(self, indexes):
        indexes = np.random.choice(self.indexes, size=self.batch_size, replace=False)
        inputs_batch = [self.inputs[i] for i in indexes]
        outputs_batch = [self.outputs[i] for i in indexes]
        
        inputs_batch = [clean_body(text) for text in inputs_batch]
        outputs_batch = [clean_highlight(text) for text in outputs_batch]


        #inputs_batch = self.inputs[idx*self.batch_size:(idx+1)*self.batch_size]
        #outputs_batch = self.outputs[idx*self.batch_size:(idx+1)*self.batch_size]
        return tokenize_inputs_and_outputs(inputs_batch, outputs_batch)

In [12]:
# Create the dataloader for the training data
train_dataset = SummaryDataset(df['total_description'][:-1000].tolist(), df['keywords'][:-1000].tolist(), batch_size=batch_size)
val_dataset = SummaryDataset(df['total_description'][-1000:].tolist(), df['keywords'][-1000:].tolist(), batch_size=batch_size)


In [13]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss=loss)


In [14]:
print(val_dataset.batch_size)
print(train_dataset.batch_size)

4
4


In [15]:
history = model.fit(train_dataset,validation_data= val_dataset , epochs= 25  )

Epoch 1/25
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert




KeyboardInterrupt: 

In [ ]:
import pickle

# Save the model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# load the saved model
with open('model2.pkl', 'rb') as f:
    model2 = pickle.load(f)

# use the model for prediction or other tasks